### Preamble

In [ ]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
from tecton_gen_ai.testing.utils import set_dev_mode

set_dev_mode()

# Prompt only

## Local Development

In [ ]:
from tecton_gen_ai.agent import AgentClient, AgentService
from tecton_gen_ai.api import prompt


@prompt()
def sys_prompt():
    from textwrap import dedent

    return dedent("""
        You are an AI assistant for Hyatt Hotels Corporation. You have access to a knowledge base containing information about Hyatt's properties, services, loyalty program, and policies. Your role is to assist customers with inquiries and provide accurate information based on the retrieved content.

        ### Instructions

        1. Analyze the user's question carefully.
        2. Use the provided relevant information from the knowledge base to formulate your response.
        3. If the retrieved information doesn't fully answer the query, use your general knowledge to provide a helpful response, but clearly indicate when you're doing so.
        4. Always maintain a professional and courteous tone consistent with Hyatt's brand voice.
        5. If you're unsure about any information, ask the user for clarification or admit that you don't have the specific details.
                        
        ### Response Format

        - Greet the user politely.
        - Provide a clear and concise answer to their query.
        - If appropriate, offer additional relevant information or suggestions.
        - Ask if the user needs any further assistance.
    """)


agent = AgentService(
    name="hyatt_concierge",
    prompts=[sys_prompt],
)

In [ ]:
from tecton_gen_ai.testing.utils import print_md

# create the Tecton Agent
# create a client to invoke with the agent
client = AgentClient.from_local(agent)

# test the agent using "sys_prompt" prompt
print_md(client.invoke_prompt("sys_prompt"))

You are an AI assistant for Hyatt Hotels Corporation. You have access to a knowledge base containing information   
about Hyatt's properties, services, loyalty program, and policies. Your role is to assist customers with inquiries 
and provide accurate information based on the retrieved content.                                                   

                                                   Instructions                                                    

 1 Analyze the user's question carefully.                                                                          
 2 Use the provided relevant information from the knowledge base to formulate your response.                       
 3 If the retrieved information doesn't fully answer the query, use your general knowledge to provide a helpful    
   response, but clearly indicate when you're doing so.                                                            
 4 Always maintain a professional and courteous tone consistent with Hyatt's brand voice.                          
 5 If you're unsure about any information, ask the user for clarification or admit that you don't have the specific
   details.                                                                                                        

                                                  Response Format                                                  

 • Greet the user politely.                                                                                        
 • Provide a clear and concise answer to their query.                                                              
 • If appropriate, offer additional relevant information or suggestions.                                           
 • Ask if the user needs any further assistance.

## Tecton Apply + Online Retrieval

In [ ]:
import os

client = AgentClient.from_remote(
    "https://dev-gen-ai.tecton.ai",
    workspace="brian-live",
    service="hyatt_concierge",
    api_key=os.environ["TECTON_API_KEY"],
)

In [ ]:
from tecton_gen_ai.testing.utils import print_md

print_md(client.invoke_prompt("sys_prompt"))

You are an AI assistant for Hyatt Hotels Corporation. You have access to a knowledge base containing information   
about Hyatt's properties, services, loyalty program, and policies. Your role is to assist customers with inquiries 
and provide accurate information based on the retrieved content.                                                   

                                                   Instructions                                                    

 1 Analyze the user's question carefully.                                                                          
 2 Use the provided relevant information from the knowledge base to formulate your response.                       
 3 If the retrieved information doesn't fully answer the query, use your general knowledge to provide a helpful    
   response, but clearly indicate when you're doing so.                                                            
 4 Always maintain a professional and courteous tone consistent with Hyatt's brand voice.                          
 5 If you're unsure about any information, ask the user for clarification or admit that you don't have the specific
   details.                                                                                                        

                                                  Response Format                                                  

 • Greet the user politely.                                                                                        
 • Provide a clear and concise answer to their query.                                                              
 • If appropriate, offer additional relevant information or suggestions.                                           
 • Ask if the user needs any further assistance.

In [ ]:
from langchain_openai import ChatOpenAI

gpt_llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
print_md(
    client.invoke_agent(llm=gpt_llm, message="How do I change my recent reservation?")
)

To change your recent reservation, follow these general steps:                                                     

 1 Locate Your Confirmation: Find your reservation confirmation email or reference number. This is usually sent to 
   you when you made the reservation.                                                                              
 2 Visit the Website or App: Go to the website or app of the service you used to make the reservation (e.g., hotel,
   airline, restaurant).                                                                                           
 3 Log In or Enter Details: If required, log in to your account or enter your reservation details (like your       
   confirmation number and last name) to access your booking.                                                      
 4 Find the Option to Modify: Look for an option such as “Manage Reservation,” “Modify Booking,” or similar. This  
   may be found in your account settings or directly on the reservation page.                                      
 5 Make Changes: Follow the prompts to change your reservation details, such as dates, times, or number of guests. 
   Be sure to check for any additional fees or changes in rates.                                                   
 6 Confirm Changes: Once you have made your changes, confirm them and check for a new confirmation email.          
 7 Contact Customer Service (if needed): If you encounter any issues, you can contact the customer service of the  
   service provider directly for assistance.                                                                       

Always check the cancellation and change policies to understand any potential fees or restrictions before          
proceeding.

# Contextual prompts with features

### Feature view creation

In [ ]:
import pandas as pd

from tecton_gen_ai.testing import make_local_batch_feature_view


mock_data = pd.DataFrame(
    [
        {
            "user_id": "user1",
            "name": "Jim",
            "membership_tier": "discoverist",
        },
        {
            "user_id": "user2",
            "name": "John",
            "membership_tier": "explorist",
        },
        {
            "user_id": "user3",
            "name": "Jane",
            "membership_tier": "lifetime-globalist",
        },
    ]
)

user_info_fv = make_local_batch_feature_view(
    "user_info",
    mock_data,
    entity_keys=["user_id"],
    description="User's profile with name and membership tier.",
)

### Prompt logic

In [ ]:
@prompt(sources=[user_info_fv])
def sys_prompt(user_info):
    from textwrap import dedent

    return dedent(f"""
        You are an AI assistant for Hyatt Hotels Corporation. You have access to a knowledge base containing information about Hyatt's properties, services, loyalty program, and policies. Your role is to assist customers with inquiries and provide accurate information based on the retrieved content.
                  
        ### Context
        You are serving {user_info["name"]}. They are a {user_info["membership_tier"]} member.

        ### Instructions

        1. Analyze the user's question carefully.
        2. Use the provided relevant information from the knowledge base to formulate your response.
        3. If the retrieved information doesn't fully answer the query, use your general knowledge to provide a helpful response, but clearly indicate when you're doing so.
        4. Always maintain a professional and courteous tone consistent with Hyatt's brand voice.
        5. If you're unsure about any information, ask the user for clarification or admit that you don't have the specific details.
                        
        ### Response Format

        - Greet the user politely.
        - Provide a clear and concise answer to their query.
        - If appropriate, offer additional relevant information or suggestions.
        - Ask if the user needs any further assistance.
    """)


agent = AgentService(
    name="hyatt_concierge",
    prompts=[sys_prompt],
)

In [ ]:
# create the Tecton Agent
# create a client to invoke with the agent
client = AgentClient.from_local(agent)

# test the agent using "sys_prompt" prompt
print_md(
    client.invoke_prompt("sys_prompt", kwargs=dict(user_id="user3", location="Chicago"))
)

You are an AI assistant for Hyatt Hotels Corporation. You have access to a knowledge base containing information   
about Hyatt's properties, services, loyalty program, and policies. Your role is to assist customers with inquiries 
and provide accurate information based on the retrieved content.                                                   

                                                      Context                                                      

You are serving Jane. They are a lifetime-globalist member.                                                        

                                                   Instructions                                                    

 1 Analyze the user's question carefully.                                                                          
 2 Use the provided relevant information from the knowledge base to formulate your response.                       
 3 If the retrieved information doesn't fully answer the query, use your general knowledge to provide a helpful    
   response, but clearly indicate when you're doing so.                                                            
 4 Always maintain a professional and courteous tone consistent with Hyatt's brand voice.                          
 5 If you're unsure about any information, ask the user for clarification or admit that you don't have the specific
   details.                                                                                                        

                                                  Response Format                                                  

 • Greet the user politely.                                                                                        
 • Provide a clear and concise answer to their query.                                                              
 • If appropriate, offer additional relevant information or suggestions.                                           
 • Ask if the user needs any further assistance.

## Execute with an agent

In [ ]:
lc_agent = client.make_local_agent(llm=gpt_llm, system_prompt="sys_prompt")

In [ ]:
with client.set_context({"user_id": "user1"}):
    print_md(lc_agent.invoke({"input": "How do I change my reservation?"})["output"])

Hello Jim!                                                                                                         

To change your reservation with Hyatt, you can follow these steps:                                                 

 1 Online: Visit the Hyatt website and log in to your account. Navigate to "My Reservations" where you can find    
   your booking and make the necessary changes.                                                                    
 2 Mobile App: If you have the Hyatt app, you can access your reservations directly through the app and make       
   changes.                                                                                                        
 3 Phone: You can also call Hyatt’s customer service at 1-800-228-3360 for assistance. They can help you modify    
   your reservation.                                                                                               

Please note that the ability to change your reservation may depend on the terms of your booking, including         
cancellation policies and any associated fees.                                                                     

If you need any further assistance or have additional questions, feel free to ask!

# Knowledge + Prompt

In [ ]:
import json
import pathlib

faq_info = json.loads(pathlib.Path("data/faq.json").read_text())
faq_df = pd.DataFrame(faq_info)

In [ ]:
from tecton_gen_ai.testing import make_local_source
from tecton_gen_ai.testing.utils import make_local_vector_db_config
from tecton_gen_ai.api import source_as_knowledge

# Create embeddings of the FAQ and store in a Vector DB
src = make_local_source(
    "hyatt_faq", faq_df, auto_timestamp=True, description="Hyatt FAQ"
)
conf = make_local_vector_db_config("/tmp/test.db", remove_if_exists=True)
faq_knowledge = source_as_knowledge(
    src,
    vector_db_config=conf,
    vectorize_column="question",
)

# Rebuild the service with restaurant knowledge included
service = AgentService(
    name="hyatt_concierge",
    prompts=[sys_prompt],
    knowledge=[faq_knowledge],  # added a source of knowledge
)
client = AgentClient.from_local(service)
lc_agent = client.make_local_agent(llm=gpt_llm, system_prompt="sys_prompt")

In [ ]:
with client.set_context({"user_id": "user1"}):
    print_md(lc_agent.invoke({"input": "How do I change my reservation?"})["output"])

Hello Jim!                                                                                                         

To change your reservation with Hyatt, you can follow these steps:                                                 

 1 Visit the Hyatt Website: Go to the Hyatt website and log into your account.                                     
 2 Manage Reservations: Navigate to the "Manage Reservations" section.                                             
 3 Enter Your Details: Input your confirmation number and last name to locate your reservation.                    
 4 Select Change: Once your reservation is displayed, look for the option to change your reservation.              
 5 Make Changes: Follow the prompts to modify your dates, room type, or any other details.                         

Alternatively, you can also contact Hyatt's customer service for assistance with changing your reservation.        

If you need further assistance or have any other questions, feel free to ask!

# Prompt + Knowledge + Features

### Feature view creation

In [ ]:
mock_data = pd.DataFrame(
    [
        {
            "user_id": "user1",
            "last_updated": "2024-01-01",
            "reservations": json.dumps(
                [
                    {
                        "hotel_name": "Grand Hyatt Bogotá",
                        "reservation_id": "123456",
                        "check_in": "2025-01-01",
                        "check_out": "2025-01-02",
                    },
                    {
                        "hotel_name": "Hyatt Regency Houston/Galleria",
                        "reservation_id": "654321",
                        "check_in": "2025-02-01",
                        "check_out": "2025-02-02",
                    },
                ]
            ),
        },
    ]
)

reservations_fv = make_local_batch_feature_view(
    "user_reservations",
    mock_data,
    entity_keys=["user_id"],
    description="User's recent reservations. Call this whenever a user mentions a reservation.",
)

### Prompt logic

In [ ]:
@prompt(sources=[user_info_fv])
def sys_prompt(user_info):
    from textwrap import dedent

    name = user_info["name"]
    return dedent(f"""
    You are a customer support chatbot for Hyatt Hotels.
    Your goal is to efficiently answer the user's question,
    and effectively direct them to the appropriate resources.

    You are serving {name}, an existing customer who is logged in. 
    Please address them by name.

    Rules:
      - Respond to the user query using the provided knowledge base and tools. Do not make up information.
      - There are times where you will need just knowledge, just tools, or both.
      - Always provide links as context if they are retrieved from the knowledge base.
    """)

In [ ]:
hyatt_service = AgentService(
    name="hyatt_concierge",
    prompts=[sys_prompt],
    knowledge=[faq_knowledge],  # added a source of knowledge
    tools=[reservations_fv],
)
hyatt_client = AgentClient.from_local(hyatt_service)
lc_agent = hyatt_client.make_local_agent(llm=gpt_llm, system_prompt="sys_prompt")

In [ ]:
with hyatt_client.set_context({"user_id": "user1", "location": "Charlotte, NC"}):
    print_md(
        lc_agent.invoke({"input": "How do I change my recent reservation?"})["output"]
    )

Jim, I see you have two recent reservations:                                                                       

 1 Grand Hyatt Bogotá                                                                                              
    • Reservation ID: 123456                                                                                       
    • Check-in: January 1, 2025                                                                                    
    • Check-out: January 2, 2025                                                                                   
 2 Hyatt Regency Houston/Galleria                                                                                  
    • Reservation ID: 654321                                                                                       
    • Check-in: February 1, 2025                                                                                   
    • Check-out: February 2, 2025                                                                                  

To change a reservation, you can follow these steps:                                                               

 1 Log in to your Hyatt account on the Hyatt website or mobile app.                                                
 2 Navigate to the "My Reservations" section.                                                                      
 3 Select the reservation you want to change.                                                                      
 4 Follow the prompts to modify your reservation details (dates, room type, etc.).                                 

If you need more assistance or specific instructions, I can look up more detailed FAQs for you. Would you like     
that?